In [1]:
import pandas as pd
import numpy as np
import os

In [7]:
import pandas as pd
import numpy as np
import os

class TradingStrategyBacktest:
    def __init__(self, data_dir='crypto_data', starting_balance=1000, risk_per_trade=0.02, take_profit=0.1):
        self.data_dir = data_dir
        self.starting_balance = starting_balance
        self.risk_per_trade = risk_per_trade
        self.take_profit = take_profit
        self.current_balance = starting_balance
        self.trades = []

    def load_data(self):
        self.data = {}
        for filename in os.listdir(self.data_dir):
            if filename.endswith('.csv'):
                symbol = filename.split('.')[0]
                filepath = os.path.join(self.data_dir, filename)
                self.data[symbol] = pd.read_csv(filepath)

    def calculate_indicators(self):
        for df in self.data.values():
            # Calculate RSI
            df['rsi'] = self.calculate_rsi(df['close'], 14)
            
            # Calculate ATR
            df['atr'] = self.calculate_atr(df['high'], df['low'], df['close'], 14)
            
            # Calculate MACD
            df['macd'], df['signal'], df['histogram'] = self.calculate_macd(df['close'])

    def calculate_rsi(self, prices, window=14):
        delta = prices.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window).mean()
        avg_loss = loss.rolling(window).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def calculate_atr(self, high, low, close, window=14):
        tr1 = pd.DataFrame(high - low).abs()
        tr2 = pd.DataFrame(high - close.shift(1)).abs()
        tr3 = pd.DataFrame(close.shift(1) - low).abs()
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(window).mean()
        return atr

    def calculate_macd(self, prices, fast=12, slow=26, signal=9):
        ema_fast = prices.ewm(span=fast, adjust=False).mean()
        ema_slow = prices.ewm(span=slow, adjust=False).mean()
        macd = ema_fast - ema_slow
        signal_line = macd.ewm(span=signal, adjust=False).mean()
        histogram = macd - signal_line
        return macd, signal_line, histogram

    def run_backtest(self):
        self.load_data()
        self.calculate_indicators()
    
        for symbol, df in self.data.items():
            print(f"Running backtest for {symbol}...")
            self.current_balance = self.starting_balance
            position = None
    
            for i in range(len(df)):
                timestamp = df.iloc[i]['timestamp']
                close = df.iloc[i]['close']
                rsi = df.iloc[i]['rsi']
                atr = df.iloc[i]['atr']
                macd = df.iloc[i]['macd']
                signal = df.iloc[i]['signal']
                histogram = df.iloc[i]['histogram']
    
                if position is None:
                    # No active position, check for entry signals
                    # Reverse the conditions for long and short entries
                    if rsi > 70 and macd < signal and histogram < 0:
                        # Enter long position (previously short entry condition)
                        position_size = self.current_balance * self.risk_per_trade / atr
                        position = {'type': 'long', 'entry': close, 'stop_loss': close * (1 - 0.03), 'take_profit': close * (1 + self.take_profit), 'size': position_size}
                        print(f"{timestamp}: Entered long position for {symbol} at {close}")
                    elif rsi < 30 and macd > signal and histogram > 0:
                        # Enter short position (previously long entry condition)
                        position_size = self.current_balance * self.risk_per_trade / atr
                        position = {'type': 'short', 'entry': close, 'stop_loss': close * (1 + 0.03), 'take_profit': close * (1 - self.take_profit), 'size': position_size}
                        print(f"{timestamp}: Entered short position for {symbol} at {close}")
                else:
                    # Active position, check for exit signals
                    if position['type'] == 'long':
                        if close <= position['stop_loss']:
                            # Stop loss hit, close position
                            pnl = (close - position['entry']) * 100 / position['entry']
                            self.current_balance -= abs(pnl) * position['size']
                            self.trades.append({'symbol': symbol, 'type': 'long', 'entry': position['entry'], 'exit': close, 'pnl': pnl, 'cost': abs(pnl) * position['size']})
                            position = None
                            print(f"{timestamp}: Closed long position for {symbol} at {close} with {pnl:.2f}% loss")
                        elif close >= position['take_profit']:
                            # Take profit hit, close position
                            pnl = (close - position['entry']) * 100 / position['entry']
                            self.current_balance += pnl * position['size']
                            self.trades.append({'symbol': symbol, 'type': 'long', 'entry': position['entry'], 'exit': close, 'pnl': pnl, 'cost': abs(pnl) * position['size']})
                            position = None
                            print(f"{timestamp}: Closed long position for {symbol} at {close} with {pnl:.2f}% profit")
                    elif position['type'] == 'short':
                        if close >= position['stop_loss']:
                            # Stop loss hit, close position
                            pnl = (position['entry'] - close) * 100 / position['entry']
                            self.current_balance -= abs(pnl) * position['size']
                            self.trades.append({'symbol': symbol, 'type': 'short', 'entry': position['entry'], 'exit': close, 'pnl': pnl, 'cost': abs(pnl) * position['size']})
                            position = None
                            print(f"{timestamp}: Closed short position for {symbol} at {close} with {pnl:.2f}% loss")
                        elif close <= position['take_profit']:
                            # Take profit hit, close position
                            pnl = (position['entry'] - close) * 100 / position['entry']
                            self.current_balance += pnl * position['size']
                            self.trades.append({'symbol': symbol, 'type': 'short', 'entry': position['entry'], 'exit': close, 'pnl': pnl, 'cost': abs(pnl) * position['size']})
                            position = None
                            print(f"{timestamp}: Closed short position for {symbol} at {close} with {pnl:.2f}% profit")
    
        # Print the final balance and trade summary
        print(f"Final balance: ${self.current_balance:.2f}")
        print("Trade summary:")
        trades_df = pd.DataFrame(self.trades)
        print(trades_df)


In [8]:
if __name__ == "__main__":
    backtest = TradingStrategyBacktest()
    backtest.run_backtest()

Running backtest for BNBUSDT...
2022-01-01 06:48:00: Entered short position for BNBUSDT at 515.2
2022-01-02 21:41:00: Closed short position for BNBUSDT at 530.7 with -3.01% loss
2022-01-02 21:42:00: Entered long position for BNBUSDT at 530.6
2022-01-03 21:00:00: Closed long position for BNBUSDT at 511.3 with -3.64% loss
2022-01-03 21:17:00: Entered short position for BNBUSDT at 510.3
2022-01-05 22:34:00: Closed short position for BNBUSDT at 458.1 with 10.23% profit
2022-01-06 07:27:00: Entered long position for BNBUSDT at 471.4
2022-01-07 03:42:00: Closed long position for BNBUSDT at 453.9 with -3.71% loss
2022-01-07 04:00:00: Entered short position for BNBUSDT at 454.1
2022-01-12 11:40:00: Closed short position for BNBUSDT at 470.1 with -3.52% loss
2022-01-12 21:13:00: Entered long position for BNBUSDT at 487.2
2022-01-14 10:55:00: Closed long position for BNBUSDT at 472.1 with -3.10% loss
2022-01-14 12:43:00: Entered long position for BNBUSDT at 473.6
2022-01-19 07:06:00: Closed long